In [8]:
import os
from math import floor
import math
import torch
import torchvision
from wildlife_tools.data.dataset import WildlifeDataset
from wildlife_datasets.datasets import MacaqueFaces, Cows2021, LeopardID2022, Cows2021v2
import torchvision.transforms as T
from wildlife_datasets import datasets, loader, metrics
from wildlife_datasets import splits
import os
from math import floor
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split
import kagglehub

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("picekl/czechlynx")

# print("Path to dataset files:", path)

100%|██████████| 11.4G/11.4G [01:19<00:00, 154MB/s] 

Extracting files...


Path to dataset files: /home/wellvw12/.cache/kagglehub/datasets/picekl/czechlynx/versions/6


In [9]:
Cows2021v2.get_data('Cows')

DATASET Cows2021v2: DOWNLOADING STARTED.


4vnrca7qw1642qlwxjadp87h7.zip: 0.00B [00:00, ?B/s]


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)>

In [ ]:
df1 = pd.read_csv('/home/wellvw12/baselines/baseline3.3.3/0/query.csv')
df2 = pd.read_csv('/home/wellvw12/baselines/baseline3.3.3/1/gallery.csv')

# missing_ids = df1[~df1['identity'].isin(df2['identity'])]['identity'].unique()

# if len(missing_ids) == 0:
#     print("All query identities exist in the gallery.")
# else:
#     print(f"Missing IDs in gallery: {missing_ids}")

df1.equals(df2)

True

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os
from math import floor
import math
import torch
import torchvision
from wildlife_tools.data.dataset import WildlifeDataset
from wildlife_datasets.datasets import MacaqueFaces, Cows2021v2, LeopardID2022, HyenaID2022
import torchvision.transforms as T
from wildlife_datasets import datasets, loader, metrics
from wildlife_datasets import splits
import os
from math import floor
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split

N_CLIENTS = 4  # 6 clients + 1 test set
OUTPUT_DIR = "/home/wellvw12/lep_4/clients"
DIRICHLET_ALPHA = 0.5
MIN_SAMPLES_PER_CLIENT = 90
TEST_SIZE = 0.15
QUERY_RATIO = 0.2  # Ratio of query samples to gallery samples
MIN_GALLERY_PER_QUERY = 2
MAX_QUERIES_PER_ID = 8
KD_TARGET_SAMPLES = 350  # Total target samples for KD set
MIN_KD_SAMPLES_PER_ID = 3
MAX_KD_SAMPLES_PER_ID = 10
KD_ID_RATIO = 0.3  # Percentage of identities to include in KD set
SEPERATE_TEST = 0.085
DIRICHLET_ALPHAS = [0.25, 0.4, 0.5, 0.9] 

def check_identity_mapping(query_set, gallery_set, metadata_path):
    """Verify consistent identity indexing between query and gallery sets."""
    metadata = LeopardID2022(metadata_path)
    # metadata = HyenaID2022(metadata_path)
    q = WildlifeDataset(query_set, metadata.root)
    g = WildlifeDataset(gallery_set, metadata.root)
    
    q_identities = q.labels_map
    g_identities = g.labels_map
    
    # Check 1: Verify all query identities exist in gallery
    unique_q = np.unique(q_identities)
    unique_g = np.unique(g_identities)
    missing = set(unique_q) - set(unique_g)
    
    if missing:
        print(f"Error: {len(missing)} query identities missing from gallery")
        print("First 5 missing:", list(missing)[:5])
        return False, {"missing_identities": list(missing)}
    
    # Check 2: Verify index positions match
    mismatches = {}
    q_indices = {id_: np.where(q_identities == id_)[0] for id_ in unique_q}
    g_indices = {id_: np.where(g_identities == id_)[0] for id_ in unique_g}
    
    for id_ in set(unique_q) & set(unique_g):
        if not np.array_equal(q_indices[id_], g_indices[id_]):
            mismatches[id_] = {
                "query_indices": q_indices[id_].tolist(),
                "gallery_indices": g_indices[id_].tolist()
            }
    
    if mismatches:
        print(f"Error: {len(mismatches)} identities have index mismatches")
        for id_, idx in list(mismatches.items())[:3]:
            print(f"{id_}:\n  Query positions: {idx['query_indices']}\n  Gallery positions: {idx['gallery_indices']}")
        return False, {"index_mismatches": mismatches}
    
    # Check 3: Verify no duplicates
    q_dupes = [id_ for id_ in unique_q if len(q_indices[id_]) > 1]
    g_dupes = [id_ for id_ in unique_g if len(g_indices[id_]) > 1]
    
    if q_dupes or g_dupes:
        print("Error: Duplicate identity indices found")
        if q_dupes: print(f"Query duplicates ({len(q_dupes)}):", q_dupes[:5])
        if g_dupes: print(f"Gallery duplicates ({len(g_dupes)}):", g_dupes[:5])
        return False, {"query_duplicates": q_dupes, "gallery_duplicates": g_dupes}
    
    print("Success: Identity mappings are consistent")
    print(f"Query identities: {len(unique_q)}")
    print(f"Gallery identities: {len(unique_g)}")
    print(f"Shared identities: {len(set(unique_q) & set(unique_g))}")
    
    return True, {
        "query_identities": len(unique_q),
        "gallery_identities": len(unique_g),
        "shared_identities": len(set(unique_q) & set(unique_g))
    }

def create_query_gallery_splits(
    df, 
    max_queries, 
    min_gallery, 
    min_samples, 
    query_ratio=QUERY_RATIO, 
    random_state=42
):
    """
    Create query/gallery splits with guaranteed gallery support.
    Tries to ensure queries make up about `query_ratio` of gallery samples.
    """
    results = {'query': [], 'gallery': []}
    id_counts = df['identity'].value_counts()
    for identity, count in id_counts.items():
        samples = df[df['identity'] == identity]
        possible_queries = min(
            max_queries, 
            max(1, int(count * query_ratio))
        )
        if count >= min_samples and possible_queries > 0 and (count - possible_queries) >= min_gallery:
            query_samples = samples.sample(possible_queries, random_state=random_state)
            results['query'].append(query_samples)
            results['gallery'].append(samples.drop(query_samples.index))
        else:
            results['gallery'].append(samples)
    query_df = pd.concat(results['query']) if results['query'] else pd.DataFrame()
    gallery_df = pd.concat(results['gallery'])
    gallery_df = gallery_df[~gallery_df.index.isin(query_df.index)]
    query_df = query_df[query_df['identity'].isin(gallery_df['identity'])]
    return query_df, gallery_df

def process_test_set(test_set_dir, metadata_path='/home/wellvw12/leopard'):
    """
    Process an individual test set directory to create validated query/gallery splits.
    
    Args:
        test_set_dir: Path to directory containing test set CSV
        metadata_path: Path to dataset root for WildlifeDataset initialization
    
    Returns:
        tuple: (success_status, report_dict)
    """
    # Path setup
    test_csv_path = os.path.join(test_set_dir, 'test.csv')
    output_query_path = os.path.join(test_set_dir, 'query.csv')
    output_gallery_path = os.path.join(test_set_dir, 'gallery.csv')
    
    if not os.path.exists(test_csv_path):
        print(f"Error: test.csv not found in {test_set_dir}")
        return False, {"error": "test.csv not found"}
    
    try:
        # Load test set
        test_df = pd.read_csv(test_csv_path)
        
        # Create query/gallery splits
        query_df, gallery_df = create_query_gallery_splits(test_df)
        
        # Verify identity mappings
        print(f"\nVerifying identity mappings for {test_set_dir}:")
        is_consistent, verification_report = check_identity_mapping(
            query_df, gallery_df, metadata_path
        )
        
        if not is_consistent:
            print("Aborting due to identity mapping issues")
            return False, verification_report
        
        # Save the validated splits
        query_df.to_csv(output_query_path, index=False)
        gallery_df.to_csv(output_gallery_path, index=False)
        
        # Print summary
        print(f"\nSuccessfully processed {test_set_dir}:")
        print(f"  Query samples: {len(query_df)}")
        print(f"  Gallery samples: {len(gallery_df)}")
        print(f"  Query identities: {len(query_df['identity'].unique())}")
        print(f"  Gallery identities: {len(gallery_df['identity'].unique())}")
        
        return True, {
            "query_samples": len(query_df),
            "gallery_samples": len(gallery_df),
            "query_identities": len(query_df['identity'].unique()),
            "gallery_identities": len(gallery_df['identity'].unique()),
            "verification_report": verification_report
        }
        
    except Exception as e:
        print(f"Error processing {test_set_dir}: {str(e)}")
        return False, {"error": str(e)}

# Load dataset
d = LeopardID2022('/home/wellvw12/leopard')
# d = HyenaID2022('/home/wellvw12/hyenaid2022')
df = d.df[~(d.df['identity'] == 'unknown')]

# Parameters


# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

def save_dataframe(df, path):
    """Safely save DataFrame, creating empty file if DataFrame is empty"""
    if len(df) == 0:
        df.head(0).to_csv(path, index=False)
    else:
        df.to_csv(path, index=False)

# --- Step 1: First split into Test and Non-Test ---
print("Creating test set...")
test_identities = np.random.choice(
    df['identity'].unique(), 
    size=int(len(df['identity'].unique()) * SEPERATE_TEST),
    replace=False
)
test_df = df[df['identity'].isin(test_identities)]
non_test_df = df[~df['identity'].isin(test_identities)]

# --- Step 2: Create Balanced KD Set ---
print("\nCreating KD set...")
def create_balanced_kd_set(df, target_samples=KD_TARGET_SAMPLES, 
                          min_samples=MIN_KD_SAMPLES_PER_ID, 
                          max_samples=MAX_KD_SAMPLES_PER_ID,
                          id_ratio=KD_ID_RATIO):
    # Select identities for KD set (prioritize those with more samples)
    id_counts = df['identity'].value_counts()
    kd_candidate_ids = id_counts[id_counts >= min_samples].index
    n_kd_ids = max(int(len(id_counts) * id_ratio), 30)  # At least 30 identities
    
    # Sort by sample count and take top identities that give us target_samples
    sorted_ids = id_counts.sort_values(ascending=False).index
    selected_ids = []
    total_samples = 0
    
    for identity in sorted_ids:
        if identity in kd_candidate_ids and len(selected_ids) < n_kd_ids:
            group = df[df['identity'] == identity]
            samples = min(max_samples, len(group))
            if total_samples + samples <= target_samples or len(selected_ids) < 30:
                selected_ids.append(identity)
                total_samples += samples
    
    # Now sample from selected identities
    kd_samples = []
    remaining_samples = []
    
    for identity in df['identity'].unique():
        group = df[df['identity'] == identity]
        if identity in selected_ids:
            n_samples = min(max_samples, max(min_samples, len(group)))
            kd_samples.append(group.sample(n=n_samples, random_state=42))
            remaining_samples.append(group.drop(kd_samples[-1].index))
        else:
            remaining_samples.append(group)
    
    kd_df = pd.concat(kd_samples)
    remaining_df = pd.concat(remaining_samples)
    
    print(f"KD set: {len(kd_df)} samples, {kd_df['identity'].nunique()} identities")
    print(f"Avg samples/identity: {len(kd_df)/kd_df['identity'].nunique():.1f}")
    print(f"Remaining: {len(remaining_df)} samples, {remaining_df['identity'].nunique()} identities")
    return kd_df, remaining_df

kd_df, client_df = create_balanced_kd_set(non_test_df)

# Save KD set as "client 0"
kd_dir = f"{OUTPUT_DIR}/0"
os.makedirs(kd_dir, exist_ok=True)
save_dataframe(kd_df, f"{kd_dir}/train.csv")
save_dataframe(pd.DataFrame(), f"{kd_dir}/query.csv")
save_dataframe(pd.DataFrame(), f"{kd_dir}/gallery.csv")

# --- Step 3: Distribute Remaining to Clients ---
print("\nDistributing to clients...")
all_identities = client_df['identity'].unique()
np.random.seed(42)
client_assignments = {i: [] for i in range(1, N_CLIENTS+1)}  # Clients 1-6

# ...existing code...
 # Example: 5 clients, first two are "smaller"
probs = np.random.dirichlet(DIRICHLET_ALPHAS, size=len(all_identities))

client_assignments = {i: [] for i in range(1, N_CLIENTS+1)}
for idx, identity in enumerate(all_identities):
    client_id = np.random.choice(N_CLIENTS, p=probs[idx]) + 1
    client_assignments[client_id].append(identity)
# ...existing code...

# Rebalance clients
for client_id in client_assignments:
    client_data = client_df[client_df['identity'].isin(client_assignments[client_id])]
    while len(client_data) < MIN_SAMPLES_PER_CLIENT:
        richest_client = max(client_assignments.items(), 
                           key=lambda x: len(client_df[client_df['identity'].isin(x[1])]))[0]
        transfer_ids = client_assignments[richest_client][-1:]  # Transfer one identity
        client_assignments[richest_client] = client_assignments[richest_client][:-1]
        client_assignments[client_id].extend(transfer_ids)
        client_data = client_df[client_df['identity'].isin(client_assignments[client_id])]

# --- Step 4: Save Test Set as Client 7 ---
test_dir = f"{OUTPUT_DIR}/7"
os.makedirs(test_dir, exist_ok=True)
save_dataframe(test_df, f"{test_dir}/train.csv")

# Create test query/gallery splits
test_query, test_gallery = create_query_gallery_splits(
    test_df,
    max_queries=MAX_QUERIES_PER_ID,
    min_gallery=MIN_GALLERY_PER_QUERY,
    min_samples=MIN_GALLERY_PER_QUERY+1,
    query_ratio=0.1,
    random_state=42
)
save_dataframe(test_query, f"{test_dir}/query.csv")
save_dataframe(test_gallery, f"{test_dir}/gallery.csv")

# --- Step 5: Process Each Client ---
print("\nProcessing clients...")
used_identities = set()
for client_id, identities in client_assignments.items():
    client_dir = f"{OUTPUT_DIR}/{client_id}"
    os.makedirs(client_dir, exist_ok=True)
    identities = [id_ for id_ in identities if id_ not in used_identities]
    used_identities.update(identities)
    client_data = client_df[client_df['identity'].isin(identities)]
    
    # Split into train/test (15% test)
    id_counts = client_data['identity'].value_counts()
    multi_sample_ids = id_counts[id_counts > 1].index
    
    if len(multi_sample_ids) == 0:
        train = client_data
        test = pd.DataFrame(columns=client_data.columns)
    else:
        df_multi = client_data[client_data['identity'].isin(multi_sample_ids)]
        df_single = client_data[~client_data['identity'].isin(multi_sample_ids)]
        
        if len(df_multi) < 2:
            train = client_data
            test = pd.DataFrame(columns=client_data.columns)
        else:
            try:
                train_multi, test_multi = train_test_split(
                    df_multi, test_size=TEST_SIZE, 
                    stratify=df_multi['identity'], random_state=42
                )
            except ValueError:
                train_multi, test_multi = train_test_split(
                    df_multi, test_size=TEST_SIZE, random_state=42
                )
            
            train = pd.concat([train_multi, df_single])
            test = test_multi
    
    # Create query/gallery from test set
    if len(test) > 0:
        query, gallery = create_query_gallery_splits(
            test,
            max_queries=MAX_QUERIES_PER_ID,
            min_gallery=MIN_GALLERY_PER_QUERY,
            min_samples=MIN_GALLERY_PER_QUERY+1,
            query_ratio=0.1,
            random_state=42
        )
        query = query[query['identity'].isin(gallery['identity'])]
        test_id_counts = test['identity'].value_counts()
        problematic = test_id_counts[test_id_counts == 1].index
        if len(problematic) > 0:
            train = pd.concat([train, test[test['identity'].isin(problematic)]])
            query = query[~query['identity'].isin(problematic)]
            gallery = gallery[~gallery['identity'].isin(problematic)]
        print(f"\nVerifying identity mapping for Client {client_id}:")
        is_consistent, report = check_identity_mapping(
            query, gallery, '/home/wellvw12/leopard'
        )
    else:
        query = pd.DataFrame(columns=client_data.columns)
        gallery = pd.DataFrame(columns=client_data.columns)
    save_dataframe(train, f"{client_dir}/train.csv")
    save_dataframe(query, f"{client_dir}/query.csv")
    save_dataframe(gallery, f"{client_dir}/gallery.csv")
    print(f"Client {client_id} saved - Train: {len(train)}, Query: {len(query)}, Gallery: {len(gallery)}")

# --- Verification ---
print("\nFinal Distribution Summary:")
all_identities = defaultdict(list)

for client in sorted(os.listdir(OUTPUT_DIR)):
    if not client.isdigit():
        continue
        
    client_path = f"{OUTPUT_DIR}/{client}"
    try:
        train = pd.read_csv(f"{client_path}/train.csv")
        query = pd.read_csv(f"{client_path}/query.csv") if os.path.getsize(f"{client_path}/query.csv") > 0 else pd.DataFrame()
        gallery = pd.read_csv(f"{client_path}/gallery.csv") if os.path.getsize(f"{client_path}/gallery.csv") > 0 else pd.DataFrame()
        
        identities = train['identity'].unique() if 'identity' in train.columns else []
        
        for id_ in identities:
            all_identities[id_].append(client)
    except Exception as e:
        print(f"Error processing client {client}: {str(e)}")
        continue

# Check for overlapping identities
overlaps = {k:v for k,v in all_identities.items() if len(v) > 1}
if overlaps:
    print(f"Warning: {len(overlaps)} identities shared across clients")
    print("Sample overlaps:", dict(list(overlaps.items())[:3]))
else:
    print("Success: All identities are unique to their clients")

print("\nClient Statistics:")
stats = []
for client in sorted(os.listdir(OUTPUT_DIR)):
    if not client.isdigit():
        continue
        
    client_path = f"{OUTPUT_DIR}/{client}"
    try:
        train = pd.read_csv(f"{client_path}/train.csv")
        query = pd.read_csv(f"{client_path}/query.csv") if os.path.getsize(f"{client_path}/query.csv") > 0 else pd.DataFrame()
        gallery = pd.read_csv(f"{client_path}/gallery.csv") if os.path.getsize(f"{client_path}/gallery.csv") > 0 else pd.DataFrame()
        
        stats.append({
            'Client': client,
            'Type': 'KD' if client == '0' else 'Test' if client == '7' else 'Train',
            'Identities': train['identity'].nunique() if 'identity' in train.columns else 0,
            'Train Samples': len(train),
            'Query Samples': len(query),
            'Gallery Samples': len(gallery),
            'Avg Samples/ID': round(len(train)/train['identity'].nunique(), 1) if 'identity' in train.columns and train['identity'].nunique() > 0 else 0
        })
    except Exception as e:
        print(f"Error gathering stats for client {client}: {str(e)}")

print(pd.DataFrame(stats).to_markdown(index=False))
print("\nKD Set Details:")
print(kd_df['identity'].value_counts().describe())
print("\nData preparation complete!")

Creating test set...

Creating KD set...
KD set: 350 samples, 35 identities
Avg samples/identity: 10.0
Remaining: 5770 samples, 394 identities

Distributing to clients...

Processing clients...

Verifying identity mapping for Client 1:
Success: Identity mappings are consistent
Query identities: 4
Gallery identities: 12
Shared identities: 4
Client 1 saved - Train: 512, Query: 7, Gallery: 68

Verifying identity mapping for Client 2:
Success: Identity mappings are consistent
Query identities: 17
Gallery identities: 23
Shared identities: 17
Client 2 saved - Train: 1601, Query: 29, Gallery: 212

Verifying identity mapping for Client 3:
Success: Identity mappings are consistent
Query identities: 18
Gallery identities: 31
Shared identities: 18
Client 3 saved - Train: 1096, Query: 19, Gallery: 140

Verifying identity mapping for Client 4:
Success: Identity mappings are consistent
Query identities: 26
Gallery identities: 34
Shared identities: 26
Client 4 saved - Train: 1832, Query: 33, Gallery:

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict
import random
from wildlife_datasets.datasets import HyenaID2022

# Global parameters
N_CLIENTS = 4
OUTPUT_DIR = "/home/wellvw12/hyena_6/clients"
DIRICHLET_ALPHA = 0.5
MIN_SAMPLES_PER_CLIENT = 90
TEST_SIZE = 0.25
QUERY_RATIO = 0.2
MIN_GALLERY_PER_QUERY = 2
MAX_QUERIES_PER_ID = 8
SEPERATE_TEST = 0.085
DIRICHLET_ALPHAS = [0.45, 0.45, 0.6, 0.9]
RANDOM_SEED = 42

# Set random seeds for reproducibility
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

def create_query_gallery_splits(
    df, 
    max_queries=MAX_QUERIES_PER_ID, 
    min_gallery=MIN_GALLERY_PER_QUERY, 
    min_samples=MIN_GALLERY_PER_QUERY+1, 
    query_ratio=QUERY_RATIO, 
    random_state=RANDOM_SEED
):
    """
    Create query/gallery splits ensuring query images are in gallery.
    """
    results = {'query': [], 'gallery': []}
    id_counts = df['identity'].value_counts()
    
    for identity, count in id_counts.items():
        samples = df[df['identity'] == identity]
        
        # Calculate number of query samples
        possible_queries = min(
            max_queries, 
            max(1, int(count * query_ratio))
        )
        
        # Only create queries if we have enough samples for gallery support
        if count >= min_samples and possible_queries > 0 and (count - possible_queries) >= min_gallery:
            # Sample query images
            query_samples = samples.sample(possible_queries, random_state=random_state)
            results['query'].append(query_samples)
            
            # Gallery includes ALL samples (including query images)
            results['gallery'].append(samples)
        else:
            # If not enough samples for queries, all go to gallery
            results['gallery'].append(samples)
    
    query_df = pd.concat(results['query']) if results['query'] else pd.DataFrame()
    gallery_df = pd.concat(results['gallery'])
    
    # Ensure query images are present in gallery
    if len(query_df) > 0:
        query_df = query_df[query_df['identity'].isin(gallery_df['identity'])]
    
    return query_df, gallery_df

def create_orientation_based_distribution(df, n_clients=N_CLIENTS):
    """
    Create non-IID distribution based on orientation with realistic heterogeneity.
    """
    print("Creating orientation-based non-IID distribution...")
    
    # Check if orientation column exists
    if 'orientation' not in df.columns:
        print("Error: No 'orientation' column found in dataset")
        return None
    
    # Get orientation distribution
    orientation_counts = df['orientation'].value_counts()
    print("Orientation distribution:")
    print(orientation_counts)
    
    # Define client-specific orientation distributions for strong non-IID
    client_distributions = {
        1: {  # Right-dominant client
            'right': 1000,
            'left': 200,
            'frontright': 50,
            'backright': 50
        },
        2: {  # Left-dominant client
            'left': 1000,
            'right': 200,
            'frontleft': 80,
            'backleft': 40
        },
        3: {  # Front-focused client
            'frontleft': 51,
            'frontright': 33,
            'front': 51,
            'right': 142,
            'left': 94
        },
        4: {  # Rare orientations client
            'up': 50,
            'back': 20,
            'down': 10,
            'backleft': 27,
            'backright': 31
        }
    }
    
    client_assignments = {i: [] for i in range(1, n_clients+1)}
    assigned_samples = set()
    
    # Assign samples based on orientation preferences
    for client_id, preferences in client_distributions.items():
        if client_id > n_clients:
            continue
            
        client_samples = []
        
        for orientation, target_count in preferences.items():
            if orientation in df['orientation'].values:
                # Get available samples for this orientation
                orientation_samples = df[
                    (df['orientation'] == orientation) & 
                    (~df.index.isin(assigned_samples))
                ]
                
                # Sample up to target_count or all available
                n_samples = min(target_count, len(orientation_samples))
                if n_samples > 0:
                    sampled = orientation_samples.sample(n=n_samples, random_state=RANDOM_SEED)
                    client_samples.append(sampled)
                    assigned_samples.update(sampled.index)
        
        if client_samples:
            client_data = pd.concat(client_samples)
            client_assignments[client_id] = client_data.index.tolist()
            print(f"Client {client_id}: {len(client_data)} samples")
            print(f"  Orientations: {client_data['orientation'].value_counts().to_dict()}")
    
    # Handle remaining unassigned samples
    remaining_samples = df[~df.index.isin(assigned_samples)]
    if len(remaining_samples) > 0:
        print(f"Distributing {len(remaining_samples)} remaining samples...")
        
        # Find client with fewest samples
        client_sizes = {i: len(client_assignments[i]) for i in range(1, n_clients+1)}
        smallest_client = min(client_sizes, key=client_sizes.get)
        
        client_assignments[smallest_client].extend(remaining_samples.index.tolist())
        print(f"Added {len(remaining_samples)} samples to Client {smallest_client}")
    
    return client_assignments

def save_dataframe(df, path):
    """Safely save DataFrame, creating empty file if DataFrame is empty"""
    if len(df) == 0:
        df.head(0).to_csv(path, index=False)
    else:
        df.to_csv(path, index=False)

# Load dataset
print("Loading HyenaID2022 dataset...")
d = HyenaID2022('/home/wellvw12/hyenaid2022')
df = d.df[~(d.df['identity'] == 'unknown')]

print(f"Total samples: {len(df)}")
print(f"Total identities: {df['identity'].nunique()}")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Step 1: Create global test set
print("\nCreating global test set...")
test_identities = np.random.choice(
    df['identity'].unique(), 
    size=int(len(df['identity'].unique()) * SEPERATE_TEST),
    replace=False
)
test_df = df[df['identity'].isin(test_identities)]
client_df = df[~df['identity'].isin(test_identities)]

print(f"Test set: {len(test_df)} samples, {test_df['identity'].nunique()} identities")
print(f"Client data: {len(client_df)} samples, {client_df['identity'].nunique()} identities")

# Step 2: Create orientation-based client assignments
client_assignments = create_orientation_based_distribution(client_df, N_CLIENTS)

if client_assignments is None:
    print("Error: Could not create orientation-based distribution")
    exit(1)

# Step 3: Create and save client data
print("\nCreating client datasets...")
for client_id in range(1, N_CLIENTS+1):
    client_dir = f"{OUTPUT_DIR}/{client_id}"
    os.makedirs(client_dir, exist_ok=True)
    
    # Get client data
    client_indices = client_assignments[client_id]
    client_data = client_df.loc[client_indices]
    
    # Create query/gallery splits
    query_df, gallery_df = create_query_gallery_splits(
        client_data,
        max_queries=MAX_QUERIES_PER_ID,
        min_gallery=MIN_GALLERY_PER_QUERY,
        min_samples=MIN_GALLERY_PER_QUERY+1,
        query_ratio=QUERY_RATIO,
        random_state=RANDOM_SEED
    )
    
    # Save client data
    save_dataframe(client_data, f"{client_dir}/train.csv")
    save_dataframe(query_df, f"{client_dir}/query.csv")
    save_dataframe(gallery_df, f"{client_dir}/gallery.csv")
    
    print(f"Client {client_id} saved:")
    print(f"  Train: {len(client_data)} samples, {client_data['identity'].nunique()} identities")
    print(f"  Query: {len(query_df)} samples")
    print(f"  Gallery: {len(gallery_df)} samples")
    if len(client_data) > 0:
        print(f"  Top orientations: {client_data['orientation'].value_counts().head(3).to_dict()}")

# Step 4: Create global test set with query/gallery splits
print("\nCreating global test set...")
test_dir = f"{OUTPUT_DIR}/test"
os.makedirs(test_dir, exist_ok=True)

test_query_df, test_gallery_df = create_query_gallery_splits(
    test_df,
    max_queries=MAX_QUERIES_PER_ID,
    min_gallery=MIN_GALLERY_PER_QUERY,
    min_samples=MIN_GALLERY_PER_QUERY+1,
    query_ratio=0.3,  # Higher ratio for test set
    random_state=RANDOM_SEED
)

save_dataframe(test_df, f"{test_dir}/all.csv")
save_dataframe(test_query_df, f"{test_dir}/query.csv")
save_dataframe(test_gallery_df, f"{test_dir}/gallery.csv")

print(f"Global test set saved:")
print(f"  All: {len(test_df)} samples, {test_df['identity'].nunique()} identities")
print(f"  Query: {len(test_query_df)} samples")
print(f"  Gallery: {len(test_gallery_df)} samples")

# Step 5: Verification
print("\nVerification:")

# Check for identity overlap between clients
all_client_identities = set()
identity_overlaps = []

for client_id in range(1, N_CLIENTS+1):
    client_path = f"{OUTPUT_DIR}/{client_id}/train.csv"
    if os.path.exists(client_path):
        client_train = pd.read_csv(client_path)
        client_identities = set(client_train['identity'].unique())
        
        # Check for overlap with previous clients
        overlap = all_client_identities.intersection(client_identities)
        if overlap:
            identity_overlaps.append(f"Client {client_id} overlaps: {len(overlap)} identities")
        
        all_client_identities.update(client_identities)

if identity_overlaps:
    print("Warning: Identity overlaps detected:")
    for overlap in identity_overlaps:
        print(f"  {overlap}")
else:
    print("✓ No identity overlaps between clients")

# Verify query images are in gallery
print("\nVerifying query images are in gallery:")
for client_id in range(1, N_CLIENTS+1):
    query_path = f"{OUTPUT_DIR}/{client_id}/query.csv"
    gallery_path = f"{OUTPUT_DIR}/{client_id}/gallery.csv"
    
    if os.path.exists(query_path) and os.path.exists(gallery_path):
        query_df = pd.read_csv(query_path)
        gallery_df = pd.read_csv(gallery_path)
        
        if len(query_df) > 0 and len(gallery_df) > 0:
            query_images = set(query_df['image_id'] if 'image_id' in query_df.columns else query_df.index)
            gallery_images = set(gallery_df['image_id'] if 'image_id' in gallery_df.columns else gallery_df.index)
            
            missing = query_images - gallery_images
            if missing:
                print(f"  Client {client_id}: {len(missing)} query images missing from gallery")
            else:
                print(f"  Client {client_id}: ✓ All query images in gallery")

# Final summary
print("\n" + "="*50)
print("DATA PREPARATION COMPLETE")
print("="*50)
print(f"Clients created: {N_CLIENTS}")
print(f"Global test set created: {len(test_df)} samples")
print(f"Random seed used: {RANDOM_SEED}")
print(f"Output directory: {OUTPUT_DIR}")